In [0]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from PIL import Image
from sklearn import linear_model as lm
from sklearn import model_selection as ms
from sklearn import preprocessing as prep
from sklearn import metrics as met
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn import svm
import pickle
import random
from random import randint
from keras_preprocessing.image import ImageDataGenerator

%matplotlib inline
plt.style.use ('dark_background')

# Ideja je sledeca: podaci su mi isecci originalnih slika i imaju dimenzije 64x64. Na osnovu njih treniram model
# Za svaku sliku iz trening skupa, prvo je iseckam na ovakve delove a onda trazim Valda u tim delovima
# Ako se nalazi u nekom od delova, onda postoji i u celoj slici, inace to nije slucaj
# Pored svakog isecka mogu da cuvam i odgovarajuce koordinate tog dela u originalnoj slici, te se lako pokaze gde je Valdo, ako postoji


In [0]:
! git clone https://github.com/vc1492a/Hey-Waldo.git
! rm -R Hey-Waldo/64/gen_waldo
! mkdir Hey-Waldo/64/gen_waldo

fatal: destination path 'Hey-Waldo' already exists and is not an empty directory.


In [0]:
# Generisanje novih slika Walda zarad balansiranja skupova

osobine = ImageDataGenerator (
        rotation_range = 180,
        horizontal_flip = True,
        vertical_flip = True,
        fill_mode = 'nearest')

# Generisemo samo slike walda, jer su klase neizbalansirane
gcolab_putanja_slika = os.path.join ('Hey-Waldo', '64')
gcolab_izlaz = os.path.join ('Hey-Waldo', '64', 'gen_waldo')

# Cuvam u gen_ direktorijumu
generator = osobine.flow_from_directory(
    directory = gcolab_putanja_slika,
    save_to_dir = gcolab_izlaz,
    save_prefix = 'gen',
    save_format = 'jpeg',
    target_size = (64, 64),
    color_mode = "rgb",
    batch_size = 4,
    class_mode = 'categorical',
    classes = ['waldo'],
    shuffle = True,
    seed=3
)

granica = 1300  # Optimalna granica

# Pokrenuti ako je potrebno generisati nove slike

for i in range (granica):
    generator.next()


Found 39 images belonging to 1 classes.


In [0]:
# Generisanje numpy nizova a zatim i csv fajlova koji predstavljaju ulazne podatke

gcolab_valdo_putanja = os.path.join ('Hey-Waldo', '64', 'gen_waldo', '')
gcolab_ne_valdo_putanja = os.path.join ('Hey-Waldo', '64', 'notwaldo', '')


valdo = np.array([np.array(plt.imread (gcolab_valdo_putanja + ime_slike)) for ime_slike in os.listdir(gcolab_valdo_putanja)])
ne_valdo = np.array([np.array(plt.imread (gcolab_ne_valdo_putanja + ime_slike)) for ime_slike in os.listdir(gcolab_ne_valdo_putanja)])

In [0]:
! rm -R CSV
! mkdir CSV

In [0]:
# Kreiranje dataframe-a za waldo slike

gcolab_csv_putanja = os.path.join ('CSV', '')

podaci = []

for slika in valdo:
    podaci.append (slika.flatten ('F'))

df1 = pd.DataFrame (podaci)
df1 ['Waldo'] = 1


In [0]:
# Kreiranje dataframe-a za ne waldo slike

podaci = []

for slika in ne_valdo:
    podaci.append (slika.flatten ('F'))

df2 = pd.DataFrame (podaci)
df2 ['Waldo'] = 0


In [0]:
# Objedinjavanje dataframe-ova i cuvanje

df = [df1, df2]
df = pd.concat (df)
df = df.apply(np.random.permutation, axis=0)

df.to_csv (gcolab_csv_putanja + 'allwaldo64.csv', index = False)

In [0]:
# Unosenje slika pomocu pandas

gcolab_csv_putanja = os.path.join ('CSV', '')

df = pd.read_csv (gcolab_csv_putanja + 'allwaldo64.csv')

X = df.drop ('Waldo', axis = 1)
y = df['Waldo']


In [0]:
# Deljenje na skupove za trening, test i validaciju

x_trening_valid, x_test, y_trening_valid, y_test = ms.train_test_split(X, y, train_size = 0.67, test_size=0.33, stratify = y, random_state = 3)
x_trening, x_valid, y_trening, y_valid = ms.train_test_split(x_trening_valid, y_trening_valid, train_size = 0.7, test_size = 0.3, stratify = y_trening_valid, random_state = 3)


In [0]:
# Centriranje

x_trening = x_trening / 255
x_valid = x_valid / 255
x_trening_valid = x_trening_valid / 255
x_test = x_test / 255


In [0]:
# Primena analize glavnih komponenti sa smanjivanjem dimenzionalnosti podataka
# 3000 je izabrano empirijski, nakon vise provera. Dosta smanjuje trajanje treniranja

'''
agk_tv = PCA (n_components = 3000, random_state = 3)
agk_tv.fit (x_trening)
x_trening = agk_tv.transform (x_trening)
x_valid = agk_tv.transform (x_valid)
'''

agk_tt = PCA (n_components = 3000, random_state = 3)
agk_tt.fit (x_trening_valid)
x_trening_valid = agk_tt.transform (x_trening_valid)
x_test = agk_tt.transform (x_test)

print ('Ukupna sacuvana varijansa sa 3000 glavnih komponenti je:', sum (agk_tt.explained_variance_ratio_)*100, '%')


Ukupna sacuvana varijansa sa 3000 glavnih komponenti je: 73.80170621466442 %


In [0]:
# Dobijanje najboljeg modela za model potpornih vektora pomocu ugradjenih funkcija
# koje koriste kros validaciju

mpv_model = svm.SVC()
parametri_mpv = {'C': [10**i for i in range(-3, 3)], 'gamma': [10**i for i in range(-3, 3)]}
mpv_najb = ms.GridSearchCV(mpv_model, param_grid=parametri_mpv, scoring='accuracy', cv = 3, return_train_score=True, n_jobs = -1)

mpv_najb.fit(x_trening_valid, y_trening_valid)
print ('Najbolji paramteri MPV modela:', mpv_najb.best_params_)


Najbolji paramteri MPV modela: {'C': 0.001, 'gamma': 0.001}


In [0]:
# Cuvanje dobijenog modela, nepotrebno jer imam najbolju konfiguraciju. Takodje, bolje je cuvati ceo model, ovo je samo proof of concept i vezbanje sa pickle bibliotekom

with open('MPV_model.pkl', 'wb') as fid:
    pickle.dump(mpv_najb.best_estimator_, fid)    

'''
# Ucitavanje modela
with open('MPV_model.pkl', 'rb') as fid:
    mpv_najb = cPickle.load(fid)
'''


"\n# Ucitavanje modela\nwith open('MPV_model.pkl', 'rb') as fid:\n    mpv_najb = cPickle.load(fid)\n"

In [0]:
# Isti postupak za KNN model

kns_model = KNeighborsClassifier (n_jobs = -1)
parametri_kns = {'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10]}
kns_najb = ms.GridSearchCV(kns_model, param_grid=parametri_kns, scoring='accuracy', cv = 3, return_train_score=True, n_jobs = -1)

kns_najb.fit(x_trening_valid, y_trening_valid)
print ('Najbolji paramteri KNS modela:', kns_najb.best_params_)


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Najbolji paramteri KNS modela: {'n_neighbors': 4}


In [0]:
# Cuvanje dobijenog modela, nepotrebno jer imam najbolju konfiguraciju

with open('KNS_model.pkl', 'wb') as fid:
    pickle.dump(kns_najb.best_estimator_, fid)    

'''
# Ucitavanje modela
with open('KNS_model.pkl', 'rb') as fid:
    kns_najb = cPickle.load(fid)
'''


"\n# Ucitavanje modela\nwith open('KNS_model.pkl', 'rb') as fid:\n    kns_najb = cPickle.load(fid)\n"

In [0]:
# Kreiranje modela logisticke regresije

lr_model = lm.LogisticRegression (solver = 'lbfgs', n_jobs = -1)
lr_model.fit (x_trening_valid, y_trening_valid)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
# Cuvanje dobijenog modela

with open('LR_model.pkl', 'wb') as fid:
    pickle.dump(lr_model, fid)    

'''
# Ucitavanje modela
with open('LR_model.pkl', 'rb') as fid:
    lr_model = cPickle.load(fid)
'''


"\n# Ucitavanje modela\nwith open('LR_model.pkl', 'rb') as fid:\n    lr_model = cPickle.load(fid)\n"

In [0]:
# Evaluacija rezultata

# Metod potpornih vektora

y_procena_mpv = mpv_najb.predict(x_test)
print ('Tacnost modela potpornih vektora je:', met.accuracy_score(y_test, y_procena_mpv))


# Metod K najblizih suseda

y_procena_kns = kns_najb.predict(x_test)
print ('Tacnost modela K najblizih suseda je:', met.accuracy_score(y_test, y_procena_kns))

# Metod logisticke regresije

y_procena_lr = lr_model.predict(x_test)
print ('Tacnost modela logisticke regresije je:', met.accuracy_score(y_test, y_procena_lr))


Tacnost modela potpornih vektora je: 0.5129548762736535
Tacnost modela K najblizih suseda je: 0.5027656477438137
Tacnost modela logisticke regresije je: 0.49199417758369723


In [0]:
# Klasifikacioni izvestaji

print ('Klasifikacioni izvestaj modela potpornih vektora je:\n', met.classification_report (y_test, y_procena_mpv), '\n')

print ('Klasifikacioni izvestaj modela K najblizih suseda je:\n', met.classification_report (y_test, y_procena_kns), '\n')

print ('Klasifikacioni izvestaj modela logisticke regresije je:\n', met.classification_report (y_test, y_procena_lr), '\n')


Klasifikacioni izvestaj modela potpornih vektora je:
               precision    recall  f1-score   support

           0       0.51      1.00      0.68      1762
           1       0.00      0.00      0.00      1673

    accuracy                           0.51      3435
   macro avg       0.26      0.50      0.34      3435
weighted avg       0.26      0.51      0.35      3435
 

Klasifikacioni izvestaj modela K najblizih suseda je:
               precision    recall  f1-score   support

           0       0.51      0.69      0.59      1762
           1       0.48      0.30      0.37      1673

    accuracy                           0.50      3435
   macro avg       0.50      0.50      0.48      3435
weighted avg       0.50      0.50      0.48      3435
 

Klasifikacioni izvestaj modela logisticke regresije je:
               precision    recall  f1-score   support

           0       0.50      0.52      0.51      1762
           1       0.48      0.46      0.47      1673

    accuracy

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
# Ostaje da se jos svaki od modela istrenira nad citavim skupom podataka, ali kako treniranje predugo traje izbegavam ovaj deo